
<br>
Example illustrating the application of MBAR to compute a 1D free energy profile <br>
from a series of force-clamp single-molecule experiments.<br>
REFERENCE<br>
    Woodside MT, Behnke-Parks WL, Larizadeh K, Travers K, Herschlag D, and Block SM.<br>
    Nanomechanical measurements of the sequence-dependent folding landscapes of single<br>
    nucleic acid hairpins. PNAS 103:6190, 2006.<br>


=============================================================================================<br>
IMPORTS<br>
=============================================================================================

In [1]:
import subprocess
import time
from pathlib import Path

In [2]:
import numpy as np
import matplotlib

In [3]:
matplotlib.use("Agg")
import matplotlib.pyplot as plt

In [4]:
import pymbar  # multistate Bennett acceptance ratio analysis (provided by pymbar)
from pymbar import timeseries, FES  # timeseries analysis (provided by pymbar)

Warning on use of the timeseries module: If the inherent timescales of the system are long compared to those being analyzed, this statistical inefficiency may be an underestimate.  The estimate presumes the use of many statistically independent samples.  Tests should be performed to assess whether this condition is satisfied.   Be cautious in the interpretation of the data.

****** PyMBAR will use 64-bit JAX! *******
* JAX is currently set to 32-bit bitsize *
* which is its default.                  *
*                                        *
* PyMBAR requires 64-bit mode and WILL   *
* enable JAX's 64-bit mode when called.  *
*                                        *
* This MAY cause problems with other     *
* Uses of JAX in the same code.          *
******************************************



=============================================================================================<br>
PARAMETERS<br>
=============================================================================================

In [5]:
prefix = "20R55_4T"  # for paper
# prefix = '10R50_4T'
# prefix = '25R50_4T'
# prefix = '30R50_4T'
directory = Path("processed-data")
temperature = 296.15  # temperature (in K)
nbins = 50  # number of bins for 1D FES
output_directory = Path("output")
plot_directory = Path("plots")

=============================================================================================<br>
CONSTANTS<br>
=============================================================================================

In [6]:
kB = 1.381e-23  # Boltzmann constant (in J/K)
pN_nm_to_kT = (1.0e-9) * (1.0e-12) / (kB * temperature)  # conversion from nM pN to units of kT

=============================================================================================<br>
SUBROUTINES<br>
=============================================================================================

In [7]:
def construct_nonuniform_bins(x_n, nbins):
    """Construct histogram using bins of unequal size to ensure approximately equal population in each bin.
    Parameters
    ----------
    x_n : 1D array of float
        x_n[n] is data point n
    Returns
    -------
    bin_left_boundary_i : 1D array of floats
        data in bin i will satisfy bin_left_boundary_i[i] <= x < bin_left_boundary_i[i+1]
    bin_center_i : 1D array of floats
        bin_center_i[i] is the center of bin i
    bin_width_i : 1D array of floats
        bin_width_i[i] is the width of bin i
    bin_n : 1D array of int32
        bin_n[n] is the bin index (in range(nbins)) of x_n[n]
    """

    # Determine number of samples.
    N = x_n.size

    # Get indices of elements of x_n sorted in order.
    sorted_indices = x_n.argsort()

    # Allocate storage for results.
    bin_left_boundary_i = np.zeros([nbins + 1])
    bin_center_i = np.zeros([nbins])
    bin_width_i = np.zeros([nbins])
    bin_n = np.zeros([N])

    # Determine sampled range, adding a little bit to the rightmost range to ensure no samples escape the range.
    x_min = x_n.min()
    x_max = x_n.max()
    x_max += (x_max - x_min) * 1.0e-5

    # Determine bin boundaries and bin assignments.
    for bin_index in range(nbins):
        # indices of first and last data points in this span
        first_index = int(float(N) / float(nbins) * float(bin_index))
        last_index = int(float(N) / float(nbins) * float(bin_index + 1))

        # store left bin boundary
        bin_left_boundary_i[bin_index] = x_n[sorted_indices[first_index]]

        # store assignments
        bin_n[sorted_indices[first_index:last_index]] = bin_index

    # set rightmost boundary
    bin_left_boundary_i[nbins] = x_max

    # Determine bin centers and widths
    for bin_index in range(nbins):
        bin_center_i[bin_index] = (
            bin_left_boundary_i[bin_index] + bin_left_boundary_i[bin_index + 1]
        ) / 2.0
        bin_width_i[bin_index] = (
            bin_left_boundary_i[bin_index + 1] - bin_left_boundary_i[bin_index]
        )
    return bin_left_boundary_i, bin_center_i, bin_width_i, bin_n

=============================================================================================<br>
MAIN<br>
=============================================================================================

In [8]:
def main():
    # read biasing forces for different trajectories
    filename = directory / f"{prefix}.forces"
    with open(filename) as infile:
        elements = infile.readline().split()
        K = len(elements)  # number of biasing forces
        # biasing_force_k[k] is the constant external biasing force used to collect trajectory k (in pN)
        biasing_force_k = np.zeros([K])
        for k in range(K):
            biasing_force_k[k] = float(elements[k])
    print("biasing forces (in pN) = ", biasing_force_k)

    # Determine maximum number of snapshots in all trajectories.
    filename = directory / f"{prefix}.trajectories"
    T_max = 0
    with open(filename) as f:
        for line in f:
            T_max += 1

    # Allocate storage for original (correlated) trajectories
    T_k = np.zeros([K], int)  # T_k[k] is the number of snapshots from umbrella simulation k`
    # x_kt[k,t] is the position of snapshot t from trajectory k (in nm)
    x_kt = np.zeros([K, T_max])

    # Read the trajectories.
    filename = directory / f"{prefix}.trajectories"
    print(f"Reading {filename}...")
    with open(filename) as infile:
        for line in infile:
            elements = line.split()
            for k in range(K):
                t = T_k[k]
                x_kt[k, t] = float(elements[k])
                T_k[k] += 1

    # Create a list of indices of all configurations in kt-indexing.
    mask_kt = np.zeros([K, T_max], dtype=bool)
    for k in range(K):
        mask_kt[k, 0 : T_k[k]] = True
    # Create a list from this mask.
    all_data_indices = np.where(mask_kt)

    # Construct equal-frequency extension bins
    print("binning data...")
    bin_kt = np.zeros([K, T_max], int)
    bin_left_boundary_i, bin_center_i, bin_width_i, bin_assignments = construct_nonuniform_bins(
        x_kt[all_data_indices], nbins
    )
    bin_kt[all_data_indices] = bin_assignments

    # Compute correlation times.
    N_max = 0
    g_k = np.zeros([K])
    for k in range(K):
        # Compute statistical inefficiency for extension timeseries
        g = timeseries.statistical_inefficiency(x_kt[k, 0 : T_k[k]], x_kt[k, 0 : T_k[k]])
        # store statistical inefficiency
        g_k[k] = g
        print(
            f"timeseries {k + 1:d} : g = {g:.1f}, {int(np.floor(T_k[k] / g)):.0f} "
            f"uncorrelated samples (of {T_k[k]:d} total samples)"
        )
        N_max = max(N_max, int(np.ceil(T_k[k] / g)) + 1)

    # Subsample trajectory position data.
    x_kn = np.zeros([K, N_max])
    bin_kn = np.zeros([K, N_max])
    N_k = np.zeros([K], int)
    for k in range(K):
        # Compute correlation times for potential energy and chi timeseries.
        indices = timeseries.subsample_correlated_data(x_kt[k, 0 : T_k[k]])
        # Store subsampled positions.
        N_k[k] = len(indices)
        x_kn[k, 0 : N_k[k]] = x_kt[k, indices]
        bin_kn[k, 0 : N_k[k]] = bin_kt[k, indices]

    # Set arbitrarynp.zeros for external biasing potential.
    x0_k = np.zeros([K])  # x position corresponding to zero of potential
    for k in range(K):
        x0_k[k] = x_kn[k, 0 : N_k[k]].mean()
    print("x0_k = ", x0_k)

    # Compute bias energies in units of kT.
    # u_kln[k,l,n] is the reduced (dimensionless) relative potential energy of
    # snapshot n from umbrella simulation k evaluated at umbrella l
    u_kln = np.zeros([K, K, N_max])
    for k in range(K):
        for l in range(K):
            # compute relative energy difference from sampled state to each other state
            # U_k(x) = F_k x
            # where F_k is external biasing force
            # (F_k pN) (x nm) (pN /
            # u_kln[k,l,0:N_k[k]] = - pN_nm_to_kT * (biasing_force_k[l] - biasing_force_k[k]) * x_kn[k,0:N_k[k]]
            u_kln[k, l, 0 : N_k[k]] = -pN_nm_to_kT * biasing_force_k[l] * (
                x_kn[k, 0 : N_k[k]] - x0_k[l]
            ) + pN_nm_to_kT * biasing_force_k[k] * (x_kn[k, 0 : N_k[k]] - x0_k[k])

    # DEBUG
    start_time = time.time()

    # Initialize MBAR.
    print("Running MBAR...")
    # TODO: change to u_kn inputs
    mbar = pymbar.MBAR(
        u_kln, N_k, verbose=True, relative_tolerance=1.0e-10, solver_protocol="robust"
    )

    # Compute unbiased energies (all biasing forces are zero).
    # u_n[n] is the reduced potential energy without umbrella restraints of snapshot n
    u_n = np.zeros([np.sum(N_k)])
    x_n = np.zeros([np.sum(N_k)])
    Nstart = 0
    for k in range(K):
        #    u_n[N_k[k]:N_k[k+1]] = - pN_nm_to_kT * (0.0 - biasing_force_k[k]) * x_kn[k,0:N_k[k]]
        u_n[Nstart : Nstart + N_k[k]] = 0.0 + pN_nm_to_kT * biasing_force_k[k] * (
            x_kn[k, 0 : N_k[k]] - x0_k[k]
        )
        x_n[Nstart : Nstart + N_k[k]] = x_kn[k, 0 : N_k[k]]
        Nstart += N_k[k]

    # Compute free energy profile in unbiased potential (in units of kT).
    print("Computing free energy profile...")
    fes = FES(u_kln, N_k, mbar_options=dict(solver_protocol="robust"))
    histogram_parameters = dict()
    # 1D array of parameters, one entry because 1D
    histogram_parameters["bin_edges"] = bin_left_boundary_i
    fes.generate_fes(u_n, x_n, histogram_parameters=histogram_parameters)
    results = fes.get_fes(
        bin_center_i, reference_point="from-lowest", uncertainty_method="analytical"
    )
    f_i = results["f_i"]
    df_i = results["df_i"]

    # compute estimate of FES including Jacobian term
    fes_i = f_i + np.log(bin_width_i)
    # Write out unbiased estimate of FES
    print("Unbiased FES (in units of kT)")
    print(f"{'bin':>8s} {'f':>8s} {'df':>8s} {'fes':>8s} {'width':>8s}")
    for i in range(nbins):
        print(
            f"{bin_center_i[i]:8.3f}",
            f"{f_i[i]:8.3f}",
            f"{df_i[i]:8.3f}",
            f"{fes_i[i]:8.3f}",
            f"{bin_width_i[i]:8.3f}",
        )
    filename = output_directory / "fes-unbiased.out"
    with open(filename, "w") as outfile:
        for i in range(nbins):
            outfile.write(f"{bin_center_i[i]:8.3f} {fes_i[i]:8.3f} {df_i[i]:8.3f}\n")
    fig_filename = plot_directory / f"fes-unbiased.pdf"
    fig, ax = plt.subplots()
    ax.errorbar(bin_center_i, fes_i, yerr=df_i, fmt="_ ", elinewidth=0.3)
    ax.set_title("Unbiased estimate of free energy profile")
    ax.set_ylabel("Free energy profile of mean force (kT)")
    ax.set_xlabel("Extension (nm)")
    fig.savefig(fig_filename)

    # DEBUG
    stop_time = time.time()
    elapsed_time = stop_time - start_time
    print(f"analysis took {elapsed_time:f} seconds")

    # compute observed and expected histograms at each state
    for l in range(K):
        # compute FES at state l
        Nstart = 0
        for k in range(K):
            u_n[Nstart : Nstart + N_k[k]] = u_kln[k, l, 0 : N_k[k]]
            Nstart += N_k[k]
        fes.generate_fes(u_n, x_n, histogram_parameters=histogram_parameters)
        results = fes.get_fes(
            bin_center_i, reference_point="from-lowest", uncertainty_method="analytical"
        )
        f_i = results["f_i"]
        df_i = results["df_i"]

        # compute estimate of FES including Jacobian term
        fes_i = f_i + np.log(bin_width_i)
        # center fes
        fes_i -= fes_i.mean()
        # compute probability distribution
        p_i = np.exp(-f_i + f_i.min())
        p_i /= p_i.sum()
        # compute observed histograms, filtering to within [x_min,x_max] range
        N_i_observed = np.zeros([nbins])
        dN_i_observed = np.zeros([nbins])
        for t in range(T_k[l]):
            bin_index = bin_kt[l, t]
            N_i_observed[bin_index] += 1
        N = N_i_observed.sum()
        # estimate uncertainties in observed counts
        for bin_index in range(nbins):
            dN_i_observed[bin_index] = np.sqrt(
                g_k[l] * N_i_observed[bin_index] * (1.0 - N_i_observed[bin_index] / float(N))
            )
        # compute expected histograms
        N_i_expected = float(N) * p_i
        # only approximate, since correlations df_i df_j are neglected
        dN_i_expected = np.sqrt(float(N) * p_i * (1.0 - p_i))
        # plot
        print(f"state {l:d} ({biasing_force_k[l]:f} pN)")
        for bin_index in range(nbins):
            print(
                f"{bin_center_i[bin_index]:8.3f}",
                f"{N_i_expected[bin_index]:10f}",
                f"{N_i_observed[bin_index]:10f} +- {dN_i_observed[bin_index]:10f}",
            )

        # Write out observed bin counts
        filename = output_directory / f"counts-observed-{l:d}.out"
        with open(filename, "w") as outfile:
            for i in range(nbins):
                outfile.write(
                    f"{bin_center_i[i]:8.3f} {N_i_observed[i]:16f} {dN_i_observed[i]:16f}\n"
                )

        # write out expected bin counts
        filename = output_directory / f"counts-expected-{l:d}.out"
        with open(filename, "w") as outfile:
            for i in range(nbins):
                outfile.write(
                    f"{bin_center_i[i]:8.3f} {N_i_expected[i]:16f} {dN_i_expected[i]:16f}\n"
                )

        # compute FES from observed counts
        indices = np.where(N_i_observed > 0)[0]
        fes_i_observed = np.zeros([nbins])
        dfes_i_observed = np.zeros([nbins])
        fes_i_observed[indices] = -np.log(N_i_observed[indices]) + np.log(bin_width_i[indices])
        fes_i_observed[indices] -= fes_i_observed[indices].mean()  # shift observed FES
        dfes_i_observed[indices] = dN_i_observed[indices] / N_i_observed[indices]
        # write out observed FES
        filename = output_directory / f"fes-observed-{l:d}.out"
        with open(filename, "w") as outfile:
            for i in indices:
                outfile.write(
                    f"{bin_center_i[i]:8.3f} {fes_i_observed[i]:8.3f} {dfes_i_observed[i]:8.3f}\n"
                )

        # Write out unbiased estimate of FES
        fes_i -= fes_i[indices].mean()  # shift to align with observed
        filename = output_directory / f"fes-expected-{l:d}.out"
        with open(filename, "w") as outfile:
            for i in range(nbins):
                outfile.write(f"{bin_center_i[i]:8.3f} {fes_i[i]:8.3f} {df_i[i]:8.3f}\n")

        # make plots
        biasing_force = biasing_force_k[l]
        fig_filename = plot_directory / f"fes-comparison-{l:d}.pdf"
        fig, ax = plt.subplots()
        ax.errorbar(
            bin_center_i, fes_i, yerr=df_i, fmt="x ", elinewidth=0.3, label="MBAR optimal estimate"
        )
        ax.errorbar(
            bin_center_i,
            fes_i_observed,
            yerr=dfes_i_observed,
            fmt="x ",
            elinewidth=0.3,
            label="Observed from single experiment",
        )
        ax.set_title(f"{prefix.title()} - {biasing_force:.2f} pN")
        ax.set_ylabel("Free energy of profile (kT)")
        ax.set_xlabel("Extension (nm)")
        ax.legend()
        fig.savefig(fig_filename)

In [9]:
if __name__ == "__main__":
    main()

biasing forces (in pN) =  [12.35 12.55 12.59 12.77 12.81 13.03 13.2  13.02 13.24 13.25 13.44 13.43
 13.69 13.94 14.19 14.41]
Reading processed-data/20R55_4T.trajectories...
binning data...
timeseries 1 : g = 63.8, 784 uncorrelated samples (of 50000 total samples)
timeseries 2 : g = 96.2, 519 uncorrelated samples (of 50000 total samples)
timeseries 3 : g = 71.4, 700 uncorrelated samples (of 50000 total samples)
timeseries 4 : g = 327.3, 152 uncorrelated samples (of 50000 total samples)
timeseries 5 : g = 259.1, 192 uncorrelated samples (of 50000 total samples)
timeseries 6 : g = 226.8, 220 uncorrelated samples (of 50000 total samples)
timeseries 7 : g = 214.6, 233 uncorrelated samples (of 50000 total samples)
timeseries 8 : g = 223.2, 223 uncorrelated samples (of 50000 total samples)
timeseries 9 : g = 191.8, 260 uncorrelated samples (of 50000 total samples)
timeseries 10 : g = 165.2, 302 uncorrelated samples (of 50000 total samples)
timeseries 11 : g = 141.9, 352 uncorrelated samples (


******* JAX 64-bit mode is now on! *******
*     JAX is now set to 64-bit mode!     *
*   This MAY cause problems with other   *
*      uses of JAX in the same code.     *
******************************************



x0_k =  [510.5142554  511.11755946 511.47392162 512.60985002 514.77599974
 516.50320767 518.00145745 516.56865004 519.56384197 523.29232689
 525.87586368 525.22540799 528.02853847 531.01662027 531.74526579
 531.91368397]
Running MBAR...
Computing free energy profile...
Unbiased FES (in units of kT)
     bin        f       df      fes    width
 494.896    0.000    0.000    2.770   15.964
 503.672   20.353    0.980   20.816    1.589
 504.992   24.908    0.978   24.958    1.051
 505.929   27.830    0.976   27.635    0.823
 506.689   30.368    0.976   30.006    0.696
 507.343   32.368    0.976   31.879    0.613
 507.929   34.200    0.975   33.616    0.557
 508.465   35.681    0.974   35.017    0.515
 508.968   37.515    0.975   36.806    0.492
 509.452   38.963    0.975   38.218    0.475
 509.918   40.497    0.975   39.713    0.457
 510.372   41.861    0.975   41.065    0.451
 510.820   43.247    0.975   42.437    0.445
 511.269   44.582    0.975   43.789    0.453
 511.720   46.000    0.97